<a href="https://colab.research.google.com/github/gauriwagare/Data_Science_Lab/blob/main/Practical2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, let's define a sample graph using an adjacency list representation. We'll use a dictionary where keys are nodes and values are lists of their neighbors.

In [1]:
# Sample Graph
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Sample Graph:")
for node, neighbors in graph.items():
    print(f"{node}: {neighbors}")

Sample Graph:
A: ['B', 'C']
B: ['A', 'D', 'E']
C: ['A', 'F']
D: ['B']
E: ['B', 'F']
F: ['C', 'E']


## Breadth-First Search (BFS)
BFS explores all the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level. It typically uses a queue data structure.

In [ ]:
#Implementation of BFS
from collections import deque

def bfs(graph, start_node):
    visited = set()  # To keep track of visited nodes
    queue = deque([start_node])  # Initialize queue with the start node
    visited.add(start_node)  # Mark the start node as visited

    bfs_path = []

    while queue:
        current_node = queue.popleft()  # Dequeue the current node
        bfs_path.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)  # Mark neighbor as visited
                queue.append(neighbor)  # Enqueue neighbor
    return bfs_path

# Demonstrate BFS starting from node 'A'
print("\nBFS Traversal (starting from A):")
bfs_result = bfs(graph, 'A')
print(bfs_result)

## Depth-First Search (DFS)
DFS explores as far as possible along each branch before backtracking. It typically uses a stack data structure (or recursion, which uses the call stack).

In [2]:
#Implementation of DFS
def dfs(graph, start_node):
    visited = set()  # To keep track of visited nodes
    stack = [start_node]  # Initialize stack with the start node
    dfs_path = []

    while stack:
        current_node = stack.pop()  # Pop the current node from the stack

        if current_node not in visited:
            visited.add(current_node)  # Mark as visited
            dfs_path.append(current_node)

            # Push unvisited neighbors onto the stack
            # We iterate in reverse to maintain consistent order if multiple paths exist
            for neighbor in reversed(graph[current_node]):
                if neighbor not in visited:
                    stack.append(neighbor)
    return dfs_path

# Demonstrate DFS starting from node 'A'
print("\nDFS Traversal (starting from A):")
dfs_result = dfs(graph, 'A')
print(dfs_result)


DFS Traversal (starting from A):
['A', 'B', 'D', 'E', 'F', 'C']


## A* Search Algorithm
A* is a popular pathfinding algorithm that finds the shortest path between a starting node and a goal node in a graph. It uses a heuristic function to estimate the cost from the current node to the goal node and a cost function (g) to track the cost from the start node to the current node. The algorithm prioritizes nodes with the lowest `f(n) = g(n) + h(n)` value.

In [3]:
# A* Search Algorithm
import heapq # For implementing a priority queue

def a_star_search(graph, start, goal, heuristic):
    # g_score: cost from start to current node
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0

    # f_score: g_score + heuristic estimate
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic[start]

    # priority_queue: (f_score, node)
    priority_queue = [(f_score[start], start)]

    # came_from: to reconstruct the path
    came_from = {}

    while priority_queue:
        current_f, current_node = heapq.heappop(priority_queue)

        if current_node == goal:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1] # Reverse to get path from start to goal

        for neighbor in graph[current_node]:
            # For simplicity, assume edge cost is 1 for unweighted graph
            tentative_g_score = g_score[current_node] + 1

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic[neighbor]
                heapq.heappush(priority_queue, (f_score[neighbor], neighbor))

    return None # No path found

# Define a heuristic function (example: straight-line distance to goal, could be 0 for uninformed search)
# For this simple graph, we'll assign arbitrary heuristic values.
# A good heuristic is 'admissible' (never overestimates the cost to reach the goal)
# and 'consistent' (triangle inequality property).
heuristic_values = {
    'A': 5,
    'B': 4,
    'C': 2,
    'D': 4,
    'E': 3,
    'F': 1  # F is closer to a hypothetical goal, or represents a lower estimated cost to goal
}

# Demonstrate A* search from 'A' to 'F'
print("\nA* Search Traversal (from A to F):")
a_star_path = a_star_search(graph, 'A', 'F', heuristic_values)
print(a_star_path)

# Demonstrate A* search from 'A' to 'D'
print("\nA* Search Traversal (from A to D):")
a_star_path_ad = a_star_search(graph, 'A', 'D', heuristic_values)
print(a_star_path_ad)


A* Search Traversal (from A to F):
['A', 'C', 'F']

A* Search Traversal (from A to D):
['A', 'B', 'D']


## Minimax Algorithm
The Minimax algorithm is a decision-making algorithm used in game theory, artificial intelligence, and statistics. It is used to minimize the possible loss for a worst-case (maximum loss) scenario. When used in the context of games, it determines the best move for a player, assuming that the opponent also plays optimally. It works by recursively exploring the game tree.

Key concepts:
- **Maximizing Player**: Aims to get the highest possible score.
- **Minimizing Player**: Aims to get the lowest possible score.
- **Terminal State**: A game state where the game has ended (win, loss, or draw).
- **Evaluation Function**: Assigns a score to a non-terminal game state, indicating its desirability for the maximizing player.

In [5]:
# --- Conceptual Game State and Helper Functions (for demonstration) ---

# This is a simplified representation. In a real game, 'state' would be a complex data structure.
# For simplicity, let's imagine a game where the 'state' is just a score that can be modified.
# The goal is for the 'maximizing player' to reach a high positive score,
# and for the 'minimizing player' to reach a low negative score.

def is_terminal(state, depth):
    # A conceptual terminal state: game ends if depth limit is reached or score is extreme
    return depth == 0 or abs(state) >= 10

def get_score(state):
    # A simple evaluation function: the state itself is the score
    return state

def get_possible_moves(state):
    # For a conceptual game, let's say a 'move' can either add or subtract a value
    # In a real game, this would generate actual game moves (e.g., available cells in Tic-Tac-Toe)
    return [state + 1, state - 1, state * 2] # Example operations

# --- Minimax Algorithm Implementation ---

def minimax(state, depth, is_maximizing_player):
    if is_terminal(state, depth):
        return get_score(state)

    if is_maximizing_player:
        best_value = -float('inf')
        for move in get_possible_moves(state):
            value = minimax(move, depth - 1, False) # Opponent will minimize
            best_value = max(best_value, value)
        return best_value
    else: # Minimizing player
        best_value = float('inf')
        for move in get_possible_moves(state):
            value = minimax(move, depth - 1, True) # Opponent will maximize
            best_value = min(best_value, value)
        return best_value

# --- Demonstration ---
print("\nDemonstrating Minimax with a conceptual game:")
initial_state = 0 # Starting score
search_depth = 3 # How many moves ahead to look

print(f"Initial state: {initial_state}, Search Depth: {search_depth}")

# Let's see what the maximizing player would choose from the initial state
maximizing_player_best_move_value = -float('inf')
maximizing_player_chosen_move = None

for move in get_possible_moves(initial_state):
    # Assume the maximizing player makes the first move, and then the minimizing player responds
    current_move_value = minimax(move, search_depth - 1, False) # After my move, opponent minimizes
    print(f"  If maximizing player chooses move resulting in state {move}, score for maximizing player is: {current_move_value}")
    if current_move_value > maximizing_player_best_move_value:
        maximizing_player_best_move_value = current_move_value
        maximizing_player_chosen_move = move

print(f"\nMinimax suggests: Maximizing player should choose move leading to state {maximizing_player_chosen_move} for a value of {maximizing_player_best_move_value}")

# Example for a minimizing player's turn
print("\n--- Example: If it's the minimizing player's turn from state 5 (with depth 2) ---")
min_player_initial_state = 5
min_player_search_depth = 2

minimizing_player_best_move_value = float('inf')
minimizing_player_chosen_move = None

for move in get_possible_moves(min_player_initial_state):
    current_move_value = minimax(move, min_player_search_depth - 1, True) # After my move, opponent maximizes
    print(f"  If minimizing player chooses move resulting in state {move}, score for maximizing player is: {current_move_value}")
    if current_move_value < minimizing_player_best_move_value:
        minimizing_player_best_move_value = current_move_value
        minimizing_player_chosen_move = move

print(f"\nMinimax suggests: Minimizing player should choose move leading to state {minimizing_player_chosen_move} for a value of {minimizing_player_best_move_value}")


Demonstrating Minimax with a conceptual game:
Initial state: 0, Search Depth: 3
  If maximizing player chooses move resulting in state 1, score for maximizing player is: 1
  If maximizing player chooses move resulting in state -1, score for maximizing player is: -1
  If maximizing player chooses move resulting in state 0, score for maximizing player is: 0

Minimax suggests: Maximizing player should choose move leading to state 1 for a value of 1

--- Example: If it's the minimizing player's turn from state 5 (with depth 2) ---
  If minimizing player chooses move resulting in state 6, score for maximizing player is: 12
  If minimizing player chooses move resulting in state 4, score for maximizing player is: 8
  If minimizing player chooses move resulting in state 10, score for maximizing player is: 10

Minimax suggests: Minimizing player should choose move leading to state 4 for a value of 8
